In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.float_format', '{:.3f}'.format)

In [1]:
from micom.media import minimal_medium
from binary_models.load_ternary import load_community_model
community, model_file = load_community_model()

Set parameter Username
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "leghorn"
Read LP format model from file C:\Users\pino216\AppData\Local\Temp\tmpgjd44x2v.lp
Reading time = 0.05 seconds
: 5454 rows, 12699 columns, 49791 nonzeros


In [4]:
med = minimal_medium(community, 0.8)

In [5]:
med.sort_index()

EX_adn_m        0.163833
EX_alltn_m      0.910797
EX_arg__L_m     0.530292
EX_ca2_m        0.004164
EX_cl_m         0.004164
EX_cobalt2_m    0.000020
EX_cu2_m        0.000567
EX_fe2_m        0.006602
EX_fe3_m        0.006246
EX_gmp_m        0.194582
EX_his__L_m     0.093400
EX_k_m          0.156154
EX_malttr_m     9.088411
EX_mg2_m        0.006940
EX_minohp_m     0.052078
EX_mn2_m        0.000553
EX_mobd_m       0.000006
EX_ni2_m        0.000258
EX_o2_m         0.000357
EX_so4_m        0.129455
EX_ump_m        0.264636
EX_zn2_m        0.000273
dtype: float64

## Optimize Model

In [6]:
with community as m:
    m.medium = med
    result = m.optimize(fluxes=True, pfba=True)
result

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.333333,2.399997,2492,2024
Rhodosporidium,0.333333,0.000000,2423,2076
Synechococcus,0.333333,0.000000,968,884
medium,NaN,NaN,466,466


In [ ]:
cols = result.fluxes.columns
ex_cols = [i for i in cols if i.startswith('EX_')]
result.fluxes[ex_cols].drop("medium").dropna(thresh=2,axis=1)

### Update Medium

In [7]:
m9_minimal = {
    'EX_nh4_m': 1000,

    'EX_pi_m': 1000,
    'EX_fe3_m': 1000,
    'EX_mn2_m': 1000,
    'EX_fe2_m': 1000.0,
    'EX_zn2_m': 1000,
    'EX_mg2_m': 1000,
    'EX_ca2_m': 1000,
    'EX_ni2_m': 1000.0,
    'EX_cu2_m': 1000,
    'EX_cobalt2_m': 1000.0,
    'EX_mobd_m': 1000.0,
    'EX_so4_m': 1000,
    'EX_k_m': 1000,
    'EX_o2_m': 1000,
    'EX_cl_m': 1000.0,
    'EX_na1_m': 1000,

    'EX_photon410_m': 1000.0,
    'EX_photon430_m': 1000.0,
    'EX_photon450_m': 1000.0,
    'EX_photon470_m': 1000.0,
    'EX_photon490_m': 1000.0,
    'EX_photon510_m': 1000.0,
    'EX_photon530_m': 1000.0,
    'EX_photon550_m': 1000.0,
    'EX_photon570_m': 1000.0,
    'EX_photon590_m': 1000.0,
    'EX_photon610_m': 1000.0,
    'EX_photon630_m': 1000.0,
    'EX_photon650_m': 1000.0,
    'EX_photon670_m': 1000.0,
    'EX_photon690_m': 1000.0,
}

In [8]:
with community as m:
    m.medium = m9_minimal
    result = m.optimize(fluxes=True, pfba=True)
result

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.333333,0.0,2492,2024
Rhodosporidium,0.333333,0.0,2423,2076
Synechococcus,0.333333,0.0,968,884
medium,NaN,NaN,466,466


In [9]:
from copy import deepcopy
# Set variable to become new medium
medium_to_change = deepcopy(m9_minimal)


medium_to_change["EX_nh4_m"] = 0
medium_to_change["EX_no3_m"] = 0

medium_to_change["EX_n2_m"] = 5

medium_to_change['EX_co2_m'] = 5

# Set the new medium as the model's medium
community.medium = medium_to_change

In [10]:
growth_only_co2 = community.optimize(fluxes=True, pfba=True)
growth_only_co2

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.333333,0.375856,2492,2024
Rhodosporidium,0.333333,0.000000,2423,2076
Synechococcus,0.333333,0.000000,968,884
medium,NaN,NaN,466,466


In [11]:
sol = community.cooperative_tradeoff(fraction =.5, fluxes=True,  pfba=True)

In [12]:
sol

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.333333,0.187926,2492,2024
Rhodosporidium,0.333333,0.000000,2423,2076
Synechococcus,0.333333,0.000000,968,884
medium,NaN,NaN,466,466


In [13]:
# Set variable to become new medium
med_with_gly = deepcopy(m9_minimal)

# use n2 as nitrogen source
med_with_gly["EX_nh4_m"] = 0
med_with_gly["EX_n2_m"] = 5

# use gly as carbon source (only RT should be able to use)
# co2 for syn?
med_with_gly["EX_gly_m"] = 100
med_with_gly['EX_co2_m'] = 5

# Set the new medium as the model's medium
community.medium = med_with_gly

In [14]:
growth_only_gyl = community.optimize(fluxes=True, pfba=True)
growth_only_gyl

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.333333,0.068609,2492,2024
Rhodosporidium,0.333333,0.000000,2423,2076
Synechococcus,0.333333,1.999997,968,884
medium,NaN,NaN,466,466


In [15]:
cols = growth_only_gyl.fluxes.columns
ex_cols = [i for i in cols if i.startswith('EX_')]
growth_only_gyl.fluxes[ex_cols].drop("medium").replace(0, np.nan).dropna(thresh=1,axis=1)

reaction,EX_acac_e,EX_ade_e,EX_ala__D_e,EX_ala__L_e,EX_asn__L_e,EX_asp__L_e,EX_btn_e,EX_ca2_e,EX_cbp_e,EX_cl_e,...,EX_photon690_e,EX_pi_e,EX_pyr_e,EX_ser__L_e,EX_so3_e,EX_so4_e,EX_spmd_e,EX_sucr_e,EX_urea_e,EX_zn2_e
compartment,,,,,,,,,,,,,,,,,,,,,
Azotobacter,6.977807,0.000015,-0.00286,-52.381141,-0.008684,-4.63635,-1.372173e-07,-0.000357,NaN,-0.000357,...,NaN,-0.066180,34.618915,-0.000228,-0.010804,-0.000298,NaN,NaN,NaN,-0.000023
Rhodosporidium,-6.977807,-0.000015,0.00286,52.381141,0.008684,4.63635,1.372173e-07,NaN,4.869102,NaN,...,NaN,-4.869102,-34.618915,0.000228,0.010804,-0.010804,0.000175,-2.945196,6.590661,NaN
Synechococcus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-100.0,-0.853401,NaN,NaN,NaN,-0.311465,-0.000175,2.945196,NaN,NaN


In [16]:
sol = community.cooperative_tradeoff(fraction =.5, fluxes=True, pfba=True)

In [17]:
sol

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.333333,0.364995,2492,2024
Rhodosporidium,0.333333,0.258045,2423,2076
Synechococcus,0.333333,0.411265,968,884
medium,NaN,NaN,466,466


In [19]:
cols = sol.fluxes.columns
ex_cols = [i for i in cols if i.startswith('EX_')]
sol.fluxes[ex_cols].drop("medium").replace(0, np.nan).dropna(thresh=1,axis=1).T#.to_csv('flux_ex_ternary_model_with_growth.csv')

compartment,Azotobacter,Rhodosporidium,Synechococcus
reaction,,,
EX_2obut_e,3.039744e-02,-3.039744e-02,NaN
EX_ac_e,1.202962e+00,-1.202962e+00,NaN
EX_acac_e,2.366020e+00,-2.366020e+00,NaN
EX_ade_e,1.587751e-02,-1.587751e-02,NaN
EX_ala__L_e,-1.618397e+01,1.618397e+01,NaN
EX_asn__L_e,-4.619745e-02,4.619745e-02,NaN
EX_asp__L_e,-1.044604e+00,1.044604e+00,NaN
EX_btn_e,-7.299908e-07,7.299908e-07,NaN
EX_ca2_e,-1.899801e-03,-2.146934e-04,NaN


In [20]:
community.reactions.EX_gly_m

Reaction identifier,EX_gly_m
Name,gly_m medium exchange
Memory address,0x02758c347a00
Stoichiometry,gly_m <=> Glycine <=>
GPR,
Lower bound,-100
Upper bound,999999.0


In [37]:
# only in model for biolog. No gly_c to gly_e reaction.
community.metabolites.gly_e__Synechococcus.reactions

frozenset({<Reaction EX_gly_e__Synechococcus at 0x2758be67340>})

In [36]:
community.reactions.EX_gly_e__Synechococcus

Reaction identifier,EX_gly_e__Synechococcus
Name,Glycine exchange
Memory address,0x02758be67340
Stoichiometry,gly_e__Synechococcus <=> 0.3333333333333333 gly_m Glycine <=> 0.3333333333333333 Glycine
GPR,
Lower bound,-100
Upper bound,1000.0


In [86]:
from cobra import Reaction, Metabolite


def add_gly_exporter(community_model):
    gly_export = Reaction(
        'gly_syn_to_med',
        name='gly export'
    )

    inside = community_model.metabolites.gly_c__Synechococcus
    outside = community_model.metabolites.gly_e__Synechococcus

    gly_export.add_metabolites({
        inside: -1.0,
        outside: 1
    })

    community_model.add_reactions([gly_export])

    community_model.reactions.gly_syn_to_med.global_id = 'gly_syn_to_med__Synechococcus'
    community_model.reactions.gly_syn_to_med.community_id = 'Synechococcus'

    # Force EX_co2_source_m to push only and EX_co2_m to pull only
    community_model.reactions.gly_syn_to_med.lower_bound = -100
    community_model.reactions.gly_syn_to_med.upper_bound = 100
    return community_model

comm2, _ = load_community_model()
comm2 = add_gly_exporter(comm2)

Read LP format model from file C:\Users\pino216\AppData\Local\Temp\tmpt5rzo_y8.lp
Reading time = 0.05 seconds
: 5454 rows, 12699 columns, 49791 nonzeros


In [87]:
comm2.reactions.EX_gly_e__Synechococcus.lower_bound = 10
comm2.reactions.EX_gly_e__Synechococcus.upper_bound = 100
comm2.reactions.EX_sucr_e__Synechococcus.lower_bound = 0
comm2.reactions.EX_sucr_e__Synechococcus.upper_bound = 0


In [88]:
# Set variable to become new medium
med_only_co2 = deepcopy(m9_minimal)

# use n2 as nitrogen source
med_only_co2["EX_nh4_m"] = 0
med_only_co2["EX_n2_m"] = 100

# use gly as carbon source (only RT should be able to use)
# co2 for syn?

med_only_co2['EX_co2_m'] = 100

# Set the new medium as the model's medium
comm2.medium = med_only_co2

In [89]:
sol = comm2.cooperative_tradeoff(fraction =.5, fluxes=True, pfba=True)

[15:49:50] WARNING  solver encountered an error infeasible_or_unbounded                             ]8;id=422320;file://C:\Users\pino216\Miniconda3\envs\whole-cell-gemm\lib\site-packages\micom\solution.py\solution.py]8;;\:]8;id=980676;file://C:\Users\pino216\Miniconda3\envs\whole-cell-gemm\lib\site-packages\micom\solution.py#224\224]8;;\

[15:50:14] WARNING  solver encountered an error infeasible_or_unbounded                             ]8;id=506018;file://C:\Users\pino216\Miniconda3\envs\whole-cell-gemm\lib\site-packages\micom\solution.py\solution.py]8;;\:]8;id=316067;file://C:\Users\pino216\Miniconda3\envs\whole-cell-gemm\lib\site-packages\micom\solution.py#224\224]8;;\

OptimizationError: could not get community growth rate.

In [ ]:
sol

In [ ]:
cols = sol.fluxes.columns
ex_cols = [i for i in cols if i.startswith('EX_')]
sol.fluxes[ex_cols].drop("medium").replace(0, np.nan).dropna(thresh=1,axis=1).T

In [ ]:
#sol.fluxes.to_csv("example_flux_gly_source.csv")